# JULIA MPI

First step was to load MPI on my mac.  Seems mpich and openmpi are two reasonable choices
with probably no beginner's reason to prefer one over the other. <br>

I did  <t> brew install gcc </t> first to get the gcc compiler.  I ran into problems.  
The magic thing that told me what to do was <t> brew doctor </t>.  It wanted me to type
<t> xcode-select --install </t> and when I did, all was good.  I then typed
<t> brew install mpich </t> and mpi was just working.

My first example was to reproduce <a href="http://www.mcs.anl.gov/research/projects/mpi/tutorial/mpiexmpl/src/pi/C/solution.html">
the classic mypi </a> in the notebook

In [1]:
using MPI

In [2]:
m = MPIManager(np=4)

MPI.MPIManager(4,Dict{Int64,Int64}(),Dict{Int64,Int64}(),0,`mpirun -np 4`,60,false,Condition(Any[]),false,0x2af8,IO[],#undef,#undef)

In [3]:
comm = MPI.COMM_WORLD;

In [4]:
addprocs(m)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [6]:
# Enter number of intervals, and tell every processor
# Traditional MPI would do this with a BCAST
@mpi_do m n = 10_000

In [9]:
# Let's see if the processors got it
@mpi_do m println(n)

	From worker 4:	10000
	From worker 2:	10000
	From worker 3:	10000
	From worker 5:	10000


In [18]:
# my MPI id
@mpi_do m myid = MPI.Comm_rank(comm)
@mpi_do m println(myid)

	From worker 2:	0
	From worker 4:	2
	From worker 3:	1
	From worker 5:	3


In [32]:
# Get the number of processors
@mpi_do m np=MPI.Comm_size(comm)
@mpi_do m println(np)

	From worker 4:	4
	From worker 2:	4
	From worker 3:	4
	From worker 5:	4


Compute $\int_0^1 4/(1+x^2) dx$ which evaluates to $\pi$

In [18]:
@mpi_do m (
   n = 10_000;
   comm = MPI.COMM_WORLD;
   s=0.0;
   for i= MPI.Comm_rank(comm)+1 :  MPI.Comm_size(comm) : n 
    x = (i-.5)/n 
    s += 4/(1+x^2) 
   end;
   mypi = s/n;
   our_π = MPI.Reduce(mypi, MPI.SUM, 0, comm)
)

@mpi_do m if myid==0; println(our_π); end
@mpi_do m if myid==0; println(our_π- π); end

	From worker 2:	3.141592654423124
	From worker 2:	8.333307377483834e-10
